In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm, tqdm_notebook

from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras.datasets import mnist
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
import pickle

from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

from sklearn.model_selection import train_test_split
from datetime import datetime

Using TensorFlow backend.


In [3]:
import math
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.optimizers import Adam, RMSprop
from tensorflow.python.keras.layers import Input, Dense, Embedding, Flatten, Dropout, merge, Activation, BatchNormalization, LeakyReLU
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.regularizers import l2
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import regularizers
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.python.keras import initializers
from tensorflow.python.keras.layers import add, concatenate
%matplotlib inline  
import warnings
warnings.filterwarnings('ignore')
from keras.callbacks import EarlyStopping, ModelCheckpoint
from scipy.sparse import csr_matrix
import tensorflow as tf
from tensorflow.python.keras.models import model_from_json
from sklearn import preprocessing
from keras.utils import plot_model
import time

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
%cd /content/drive/My Drive/TOBIGS/Recommender System/
base_path = os.getcwd()
input_path = os.path.join(base_path, 'input')
model_path = os.path.join(base_path, 'model')

/content/drive/My Drive/TOBIGS/Recommender System


## **Input**

In [5]:
data = pd.read_csv(os.path.join(input_path, 'input_data.csv'), index_col=0)

### Preprocessing

In [6]:
data.loc[data['place.id']==6998634]

,idd,user.id,place.id,place.name,category,place.category,user.rating,place.rating,createdDate,is_fch,photonum,review.count,language,lat,lng,land.addr,review,is_local
472244,472221,464DD2C167B49672C76FA7ABF630ED9A,6998634.0,ibis budget Ambassador Seoul Dongdaemun,ACM,NaN,4.0,3.94723,2020-03-08,0,0.0,379,ko,NaN,NaN,NaN,"전반적으로 만족스러웠던 숙박.\n객실의 청결도는 나쁘지 않았고, 기타 조건들 역시 ...",1
472245,472222,71B643D3A2337B0B61B9DF6654325DED,6998634.0,ibis budget Ambassador Seoul Dongdaemun,ACM,NaN,4.0,3.94723,2020-01-22,0,0.0,379,en,NaN,NaN,NaN,enjoyed my short stay at this hotel. good loca...,0
472246,472223,12D9D66B65B83900C51A4EABC84B00E2,6998634.0,ibis budget Ambassador Seoul Dongdaemun,ACM,NaN,5.0,3.94723,2020-01-06,0,0.0,379,th,NaN,NaN,NaN,โรงแรมสะอาด ปลอดภัย เหมาะสำหรับคนเอเชีย มีบริก...,0
472247,472224,69B69CB3702C6F0E1F73017CA4575C36,6998634.0,ibis budget Ambassador Seoul Dongdaemun,ACM,NaN,5.0,3.94723,2020-01-01,0,0.0,379,en,NaN,NaN,NaN,Stayed with family members for vacation.Good l...,0
472248,472225,5420187FEEB4A642C6ABC9676C6B8544,6998634.0,ibis budget Ambassador Seoul Dongdaemun,ACM,NaN,3.0,3.94723,2019-12-23,0,9.0,379,ja,NaN,NaN,NaN,いたって、普通。どちらかというと、狭いが、寝れればいいかな。という感じ。隣がうるさかった。安...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472618,472595,9087C800819B14D400E489B14A001ECE,6998634.0,ibis budget Ambassador Seoul Dongdaemun,ACM,NaN,4.0,3.94723,2014-11-20,0,0.0,379,en,NaN,NaN,NaN,Positives: The location of this hotel is reall...,0
472619,472596,57C1FA73133D857391422FD0B26E6C38,6998634.0,ibis budget Ambassador Seoul Dongdaemun,ACM,NaN,4.0,3.94723,2014-11-18,0,0.0,379,en,NaN,NaN,NaN,Location was great for shopping and also getti...,0
472620,472597,ADF184269F6289CCA4256EBCF8DE4C8E,6998634.0,ibis budget Ambassador Seoul Dongdaemun,ACM,NaN,4.0,3.94723,2014-11-17,0,0.0,379,en,NaN,NaN,NaN,Great new hotel. Rooms were a bit small but wa...,0
472621,472598,6417BA7615824C0F79CA40C027791FEF,6998634.0,ibis budget Ambassador Seoul Dongdaemun,ACM,NaN,3.0,3.94723,2014-11-16,0,0.0,379,ja,NaN,NaN,NaN,ツインの部屋に夫婦で３泊しました。\n地下鉄駅４番出口（階段のみ）から徒歩５分以内です。\n...,0


In [7]:
local_df = data.loc[data['is_local']==1]
global_df = data.loc[data['is_local']==0]
print(local_df.shape, global_df.shape)

(424262, 18) (48352, 18)


In [8]:
local_df.category.value_counts()

EAT    415492
ACM      8770
Name: category, dtype: int64

In [9]:
global_df.category.value_counts()

ACM    48352
Name: category, dtype: int64

In [10]:
place_name = local_df.drop_duplicates(subset=['place.id'])[['place.id', 'place.name']]
place_name_dict = {}
for id, name in zip(place_name['place.id'].values, place_name['place.name'].values):
    place_name_dict[id] = name

In [11]:
df = local_df[['user.id', 'place.id', 'user.rating', 'createdDate']]
df.createdDate = pd.to_datetime(df.createdDate)
df.columns = ['userId', 'placeId', 'rating', 'timestamp']

df.timestamp = df.timestamp.apply(lambda x: time.mktime(x.timetuple()))

In [12]:
user_dict = {}
for idx, unique_user in enumerate(df.userId.unique()):
    user_dict[unique_user] = idx
    
place_id_dict = {}
for idx, unique_location in enumerate(df.placeId.unique()):
    place_id_dict[unique_location] = idx

In [13]:
df['userId'] = df['userId'].map(user_dict)
df['placeId'] = df['placeId'].map(place_id_dict)

### Make sparse matrix

In [14]:
users = CategoricalDtype(sorted(df.userId.unique()), ordered=True)
places = CategoricalDtype(sorted(df.placeId.unique()), ordered=True)
row = df.userId.astype(users).cat.codes
col = df.placeId.astype(places).cat.codes

In [15]:
sparse_matrix = csr_matrix((df['rating'], (row, col)) , shape= (users.categories.size, places.categories.size))

In [16]:
sparse_matrix.shape

(70614, 15675)

### Split dataset

In [17]:
x_train, x_test = train_test_split(sparse_matrix, train_size=0.8, shuffle=True, random_state=2020, )

## **Model**

#### Define Loss

In [18]:
def masked_mse(y_true, y_pred):
    # masked function
    mask_true = K.cast(K.not_equal(y_true, 0), K.floatx())
    # masked squared error
    masked_squared_error = K.square(mask_true * (y_true - y_pred))
    masked_mse = K.sum(masked_squared_error, axis=-1) / K.maximum(K.sum(mask_true, axis=-1), 1)
    return masked_mse

In [19]:
def masked_rmse_clip(y_true, y_pred):
    # masked function
    mask_true = K.cast(K.not_equal(y_true, 0), K.floatx())
    y_pred = K.clip(y_pred, 1, 5)
    # masked squared error
    masked_squared_error = K.square(mask_true * (y_true - y_pred))
    masked_mse = K.sqrt(K.sum(masked_squared_error, axis=-1) / K.maximum(K.sum(mask_true, axis=-1), 1))
    return masked_mse

#### Auto Encoder

In [20]:
def deepAE(X, layers, activation, last_activation, dropout, regularizer_encode, regularizer_decode, side_infor_size=0):
    input_layer = x = Input(shape=(X.shape[1],), name='UserRating', sparse=True)
    k = int(len(layers)/2)
    i = 0
    for l in layers[:k]:
        x = Dense(l, activation=activation,
                      name='EncLayer{}'.format(i), kernel_regularizer=regularizers.l2(regularizer_encode))(x)
        i = i+1
    x = Dense(layers[k], activation=activation, 
                                name='LatentSpace', kernel_regularizer=regularizers.l2(regularizer_encode))(x)    
    x = Dropout(rate = dropout)(x)    
    for l in layers[k+1:]:
        i = i-1
        x = Dense(l, activation=activation, 
                      name='DecLayer{}'.format(i), kernel_regularizer=regularizers.l2(regularizer_decode))(x)
    output_layer = Dense(X.shape[1]-side_infor_size, activation=last_activation, name='UserScorePred', kernel_regularizer=regularizers.l2(regularizer_decode))(x)
    model = Model(input_layer, output_layer)

    return model

In [21]:
layers = [1024, 512, 258, 512, 1024]
dropout = 0.8 
activation = 'selu'
last_activation = 'selu'
regularizer_encode = 0.001
regularizer_decode = 0.001
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
epochs=10

In [22]:
# Build model
DeepAE = deepAE(x_train, layers, activation, last_activation, dropout, regularizer_encode, regularizer_decode)
DeepAE.compile(optimizer = optimizer, loss=masked_mse, metrics=[masked_rmse_clip]) 
DeepAE.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
UserRating (InputLayer)      [(None, 15675)]           0         
_________________________________________________________________
EncLayer0 (Dense)            (None, 1024)              16052224  
_________________________________________________________________
EncLayer1 (Dense)            (None, 512)               524800    
_________________________________________________________________
LatentSpace (Dense)          (None, 258)               132354    
_________________________________________________________________
dropout (Dropout)            (None, 258)               0         
_________________________________________________________________
DecLayer1 (Dense)            (None, 512)               132608    
_________________________________________________________________
DecLayer0 (Dense)            (None, 1024)              525312

In [23]:
x_train.shape[0] , x_test.shape[0]

(56491, 14123)

#### Batch generator

In [24]:
def nn_batch_generator(X_data, y_data, batch_size):
    samples_per_epoch = X_data.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    index = np.arange(np.shape(y_data)[0])
    while 1:
        index_batch = index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X_data[index_batch,:].todense()
        y_batch = y_data[index_batch].todense()
        counter += 1
        yield np.array(X_batch),y_batch
        if (counter > number_of_batches):
            counter=0

#### Load model weight

In [25]:
DeepAE.load_weights(os.path.join(model_path, './weight/0622_local_weights.h5'))

#### 학습

In [26]:
result_deepAE = DeepAE.fit_generator(nn_batch_generator(x_train, x_train, 20), steps_per_epoch=1000, epochs=epochs, verbose=2,
                                    validation_data=[x_test, x_test])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
1000/1000 - 10s - loss: 1.1585 - masked_rmse_clip: 0.7315 - val_loss: 0.2354 - val_masked_rmse_clip: 0.0000e+00
Epoch 2/10
1000/1000 - 10s - loss: 1.0614 - masked_rmse_clip: 0.6695 - val_loss: 0.2565 - val_masked_rmse_clip: 0.0000e+00
Epoch 3/10
1000/1000 - 10s - loss: 0.9253 - masked_rmse_clip: 0.6069 - val_loss: 0.2647 - val_masked_rmse_clip: 0.0000e+00
Epoch 4/10
1000/1000 - 10s - loss: 0.7770 - masked_rmse_clip: 0.5460 - val_loss: 0.2786 - val_masked_rmse_clip: 0.0000e+00
Epoch 5/10
1000/1000 - 10s - loss: 0.7746 - masked_rmse_clip: 0.5328 - val_loss: 0.2973 - val_masked_rmse_clip: 0.0000e+00
Epoch 6/10
1000/1000 - 10s - loss: 0.7118 - masked_rmse_clip: 0.4905 - val_loss: 0.2914 - val_masked_rmse_clip: 0.0000e+00
Epoch 7/10
1000/1000 - 10s - loss: 0.6779 - masked_rmse_clip: 0.4803 - val_loss: 0.2893 - val_masked_rmse_clip: 0.0000e+00
Epoch 8/10
1000/1000 - 10s - loss: 0.6802 - masked_rmse_clip: 0

In [27]:
#DeepAE.save_weights(os.path.join(model_path, './weight/0622_local_weights.h5'))

#### Result

In [28]:
pred = DeepAE.predict_on_batch(sparse_matrix)

In [29]:
pred_clip = np.where(pred < 3.5, 0, pred)
pred_clip = np.where(pred>5, 5, pred_clip)
pred = None # 메모리 없애주기

In [30]:
users_index = sorted(df.userId.unique())
places_index = sorted(df.placeId.unique())

## **SAR**

In [31]:
!pip install papermill

     |████████████████████████████████| 71kB 3.4MB/s 
     |████████████████████████████████| 102kB 5.3MB/s 
     |████████████████████████████████| 747kB 32.3MB/s 
ERROR: nbclient 0.4.0 has requirement jupyter-client>=6.1.0, but you'll have jupyter-client 5.3.4 which is incompatible.


In [32]:
!pip install git+https://github.com/microsoft/recommenders

  Cloning https://github.com/microsoft/recommenders to /tmp/pip-req-build-mfh4g5lh
  Running command git clone -q https://github.com/microsoft/recommenders /tmp/pip-req-build-mfh4g5lh
  Created wheel for reco-utils: filename=reco_utils-2019.9-cp36-none-any.whl size=164194 sha256=b4b96aeaffc7c8566abc9b09c619422af233f980615a97bc5b3bd21d181e2261
  Stored in directory: /tmp/pip-ephem-wheel-cache-7zo9mivb/wheels/b4/1c/70/c3f6005781b2b5a3370ec55bdd75c10ed1b07456bdfb1a0524
Successfully built reco-utils


In [33]:
import itertools
import logging
import os
import papermill as pm
from reco_utils.dataset.python_splitters import python_stratified_split, python_random_split, python_chrono_split
from reco_utils.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from reco_utils.recommender.sar.sar_singlenode import SARSingleNode

import datetime
import time
 #%matplotlib inline
import numpy as np
import pickle
import os
#from matplotlib import pyplot as plt
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import objectives
from keras import backend as K
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, Callback
from IPython.display import clear_output
import matplotlib.pyplot as plt
import pandas as pd

#### Make original data

In [34]:
restore_df = pd.DataFrame(pred_clip.reshape(-1,1))
pred_clip =None# 메모리 없애주기
restore_df.index = pd.MultiIndex.from_product([users_index, places_index])

In [35]:
restore_df_three = restore_df.loc[restore_df[0]>=5]
restore_df = None

In [36]:
restore_df_three.shape

(73872045, 1)

In [37]:
restore_df = restore_df_three.reset_index()
restore_df.columns = ['userId', 'placeId', 'rating']

In [38]:
input_data = pd.merge(restore_df, df, how= 'left', on=['userId', 'placeId'])
restore_df = None
df = None

In [39]:
input_data.head(3)

,userId,placeId,rating_x,rating_y,timestamp
0,0,5,5.0,5.0,1.588893e+09
1,0,7,5.0,5.0,1.588265e+09
2,0,8,5.0,5.0,1.587130e+09


In [40]:
input_data.drop('rating_y', axis=1, inplace=True)

In [41]:
input_data.timestamp.interpolate(method='values', inplace=True)
input_data.timestamp.fillna(method='bfill', inplace=True)

In [42]:
input_data.isnull().sum()

userId       0
placeId      0
rating_x     0
timestamp    0
dtype: int64

#### Feature Processing & Split dataset

In [43]:
df=input_data
df.columns = ['id','location.locationId',	'rating',	'createdDate']
input_data = None

In [44]:
# top k items to recommend
TOP_K = 100
DATA_SIZE = df.shape[0]

In [45]:
train, test = train_test_split(df[:300000], train_size=0.75, random_state=2020, shuffle=True, stratify=df['id'][:300000])

: 모든 데이터를 사용하는데 시간이 많이 걸리고 세션이 다운되기 때문에 다음과 같이 sampling하여 사용해보자.

In [46]:
train.shape, test.shape

((225000, 4), (75000, 4))

In [47]:
train.head(3)

,id,location.locationId,rating,createdDate
233460,142,4456,5.0,1.576905e+09
236396,145,3556,5.0,1.582590e+09
63963,35,7012,5.0,1.580443e+09


In [48]:
header = {
    "col_user": "id",
    "col_item": "location.locationId",
    "col_rating": "rating",
    "col_timestamp": "createdDate",
}

In [49]:
model = SARSingleNode(
    similarity_type="cooccurrence", # jaccard, lift, cooccurrence 
    time_decay_coefficient=100, 
    time_now=None, 
    timedecay_formula=True, 
    **header
)

#### Model & Output

In [50]:
start = time.time()
model.fit(train)
print(time.time()-start)

4.353102684020996


In [51]:
start = time.time()
top_k = model.recommend_k_items(test, remove_seen=True)
print(time.time()-start)

6.036222219467163


### Result

In [52]:
def find_origin_user(user_num) : 
    user_name=list(user_dict.keys())[list(user_dict.values()).index(user_num)]
    return user_name

In [53]:
def find_origin_place_id(place_num) :
    place_id=list(place_id_dict.keys())[list(place_id_dict.values()).index(place_num)]
    return place_id

In [54]:
def print_output(top_k, user_index):
    visited_place = local_df.loc[local_df['user.id']==find_origin_user(user_index), 'place.name'].to_list()
    
    recommender_id = []
    for place_index in top_k.loc[top_k['id']==user_index, 'location.locationId'].to_list():
        recommender_id.append(find_origin_place_id(place_index))

    recommend_place = local_df.loc[local_df['place.id'].isin(recommender_id) , 'place.name'].unique()
    return visited_place, recommend_place

In [55]:
visited_place , recomm_placed = print_output(top_k, 0)

In [56]:
visited_place[:10]

['스타벅스 보라매공원R점',
 '해인어시장 신림본점',
 '보라매양평해장국',
 '배영만당진아구동태찜탕 보라매본점',
 '파리바게뜨 당곡사거리점',
 '마포갈비',
 '원주골돌솥추어탕',
 '진격의곱창 신림점',
 '킷사서울',
 '빽다방 보라매타운점']

In [57]:
recomm_placed

array(['다운타우너 잠실점', '후레쉬빌', '싸다김밥 상왕십리역점', 'H3 bistro', '피자알볼로 연신내점',
       '코코이찌방야 합정역메세나폴리스점', '회식의달인 신논현점', '대장금', '농민백암왕순대', '라화쿵부 - 신촌점'],
      dtype=object)

In [58]:
visited_place , recomm_placed = print_output(top_k, 10)

In [59]:
visited_place[:10]

['스타벅스 보라매공원R점',
 '배영만당진아구동태찜탕 보라매본점',
 '파리바게뜨 당곡사거리점',
 '마티스커피',
 '경성찹쌀꽈배기',
 '더벤티 신림점',
 '이화수전통육개장 서울보라매점',
 '생각보다맛있는집 신림점',
 '굴마을낙지촌 신림점',
 '메가엠지씨커피 보라매타운점']

In [60]:
recomm_placed

array(['배진수어시장', '진대감 신사점', '마약떡볶이', '고부기 대학로본점', '교촌치킨 왕십리1호점', '영천 떡볶이',
       '대장금', '라화쿵부 - 신촌점', '이태리총각', '테이블포포'], dtype=object)

## **Recommender**

: user_base_collaborative_filtering

In [61]:
def get_input(place, user=None):
    userId = user
    placeName = place
    return [userId, placeName]

In [62]:
def find_similar_user(placeName, k=10):

    sim_user_group = []
    for placename in get_input(placeName)[1]:
        sim_user = local_df.loc[local_df['place.name'].str.contains(placename), 'user.id'].to_list()
        sim_user_group.append(sim_user)
    max_sim_user = max(sim_user_group,key=sim_user_group.count)
    
    user_index = user_dict[max_sim_user[0]] # 임의의 숫자
    visited_place, recommender = print_output(top_k, user_index)
    concat_place = visited_place + list(recommender)

    recommender_list = []
    for duplicate_place in concat_place:
        for place in placeName:
            if duplicate_place.find(place) <= -1:
                recommender_list .append(duplicate_place)
    return list(set(recommender_list))[:k]

In [63]:
input_place = ['생각보다맛있는집 신림점','굴마을낙지촌 신림점','메가엠지씨커피 보라매타운점']
k = 10

In [64]:
find_similar_user(input_place, k=10) 

['죠스떡볶이 이수역점',
 '베이크스토리 신림점',
 '담소소사골순대육개장 노량진점',
 '남도미항',
 '스타벅스 이수자이점',
 '토마토김밥 대학동점',
 '굴마을낙지촌 신림점',
 '맘스터치 대학동점',
 '전라도시네마',
 '배영만당진아구 낙성대점']

In [65]:
input_place = ['옷살', '투썸플레이스']

In [66]:
find_similar_user(input_place, k=10) 

['다케오호르몬데판야끼 송리단길점',
 '닌자스시',
 '논현삼계탕송파점',
 '옷살',
 'STAY',
 '글로우',
 '두만강샤브샤브',
 '서정적살롱',
 '연안식당 잠실점',
 '부부횟집']

In [71]:
input_place = ['옷살', '투썸플레이스', '맘스터치', '']

In [73]:
find_similar_user(input_place, k=20) 

['보칼리노',
 'Grand Mercure Ambassador Hotel and Residences Seoul Yongsan',
 'Lotte Hotel Seoul',
 'Grand InterContinental Seoul Parnas',
 'Le Meridien Seoul',
 'The Shilla Seoul',
 'Signiel Seoul',
 'Conrad Seoul',
 'Four Seasons Hotel Seoul']

In [81]:
local_df.loc[local_df['category']=='ACM']

,idd,user.id,place.id,place.name,category,place.category,user.rating,place.rating,createdDate,is_fch,photonum,review.count,language,lat,lng,land.addr,review,is_local
415515,415492,3rhAR,3.672804e+07,31 페이지,ACM,NaN,4.0,4.000000,2019-12-27 15:37:00,0,0.0,1,ko,NaN,NaN,서울특별시 동대문구 전농동 597-23,숙박하기 참 애매한 곳입니다. 기찻길이 바로 뒤에 위치하여 기차 운영시간 대에는 소...,1
415516,415493,1pTDP,3.430020e+07,Ever8레지던스,ACM,NaN,4.0,4.000000,2019-12-28 16:54:00,0,0.0,1,ko,NaN,NaN,서울특별시 서대문구 대현동 104-5,NaN,1
415517,415494,a7p4r,3.749021e+07,E호텔,ACM,NaN,2.0,2.000000,2019-08-07 18:23:20,0,0.0,1,ko,NaN,NaN,서울특별시 서초구 반포대로18길 40 E 호텔,객실 청결도는 별로고 조식이 맛있음,1
415518,415495,2waUO,9.662794e+08,G2 호텔,ACM,NaN,4.0,4.000000,2019-10-29 12:54:00,0,0.0,1,ko,NaN,NaN,서울특별시 중구 저동2가 48-27,NaN,1
415519,415496,27xyc,1.245880e+09,Hotel in 9,ACM,NaN,4.0,4.300000,2019-05-27 08:39:16,0,0.0,10,ko,NaN,NaN,서울특별시 강남구 영동대로 618 Hotel in 9,"점심에 런치뷔페 이용했는데, 음식 종류가 많지는 않았지만 대부분 다 맛있어요! 뷰는...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472628,472605,11ABDC62AA61ED621186E341AE6A6356,2.995330e+05,ibis styles Ambassador Seoul Gangnam,ACM,NaN,5.0,4.230769,2018-02-25,0,0.0,13,ko,NaN,NaN,NaN,앰배서더 회원이라 평소 그랜드 앰배서더 풀만이랑 노보텔등 위주로 자주 다닙니다..\...,1
472629,472606,99E695C357C38B43A91AC43827F53D9A,2.995330e+05,ibis styles Ambassador Seoul Gangnam,ACM,NaN,5.0,4.230769,2015-11-25,0,0.0,13,ko,NaN,NaN,NaN,"슈페리어 스위트로 1박했습니다. 넓은 객실에는 대형 티비와 쇼파, 6인용 식탁이 있...",1
472630,472607,3D81C4D92B1C58BFF0F9011FE4C08906,2.995330e+05,ibis styles Ambassador Seoul Gangnam,ACM,NaN,3.0,4.230769,2015-12-30,0,0.0,13,ko,NaN,NaN,NaN,근처에 일이 있다면 묵을만한 비즈니스 호텔입니다. \n이비스 체인답게 객실은 깨끗하...,1
472631,472608,CDE1C110B78BAAA83D7012B083D589E2,2.995330e+05,ibis styles Ambassador Seoul Gangnam,ACM,NaN,4.0,4.230769,2015-09-14,0,0.0,13,ko,NaN,NaN,NaN,좋은 위치와 맛스러운 맛집이 대박..\n공항버스가 길건너에서 탈수 있고 \n전철역에...,1
